In [1]:
import oandapyV20
import oandapyV20.endpoints.instruments as instruments
import oandapyV20.endpoints.pricing as pricing
import oandapyV20.endpoints.accounts as accounts
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.trades as trades
import pandas as pd

In [2]:
client = oandapyV20.API(access_token="5feaa802c52be3e714d4bd74bc1a9169-8cbadb5f1a922e05aad7d56841b3f44a",environment="practice")
account_id = "101-002-19512089-001"
#defining strategy parameters

In [3]:
tickers=["EUR_USD","AUD_USD","GBP_USD","USD_JPY","EUR_JPY"]
pos_size=5000                        

In [4]:
def market_order(instrument,units,sl):
    """units can be positive or negative, stop loss (in pips) added/subtracted to price """  
    account_id = "101-002-19512089-001"

    price=pricing.PricingInfo(accountID=account_id)
    client.request(price)    
    data = {
            "order": {
            "price": "",
            "stopLossOnFill": {
            "trailingStopLossOnFill": "GTC",
            "distance": str(sl)
                              },
            "timeInForce": "FOK",
            "instrument": str(instrument),
            "units": str(units),
            "type": "MARKET",
            "positionFill": "DEFAULT"
                    }
            }
    r = orders.OrderCreate(accountID=account_id, data=data)
    client.request(r)
    print("order filled for ",instrument)

In [5]:
def candles(instrument):
    params = {"count": 50,"granularity": "M1"} #granularity can be in seconds S5 - S30, minutes M1 - M30, hours H1 - H12, days D, weeks W or months M
    candles = instruments.InstrumentsCandles(instrument=instrument,params=params)
    client.request(candles)
    ohlc_dict = candles.response["candles"]
    ohlc = pd.DataFrame(ohlc_dict)
    ohlc_df = ohlc.mid.dropna().apply(pd.Series)
    ohlc_df["volume"] = ohlc["volume"]
    ohlc_df.index = ohlc["time"]
    ohlc_df = ohlc_df.apply(pd.to_numeric)
    ohlc_df.columns=["Open","High","Low","Close","Volume"]
    print("took data for ",instrument)
    return ohlc_df

In [6]:
def trade_signal(instrument,l_s):
    df=pd.DataFrame()
    df=candles(instrument)
    df["open-close"]=abs(df["Open"]-df["Close"])
    signal=""
    if l_s=="":
        if df["Volume"].tolist()[-1]>1.5*df["Volume"].rolling(20).max()[-1] and df["Open"][-1]<df["Close"][-1] and df["open-close"][-1]>df["open-close"].rolling(20).mean()[-1]:
            signal="buy"

        elif df["Volume"].tolist()[-1]>1.5*df["Volume"].rolling(20).max()[-1] and df["Open"][-1]>df["Close"][-1] and df["open-close"][-1]>df["open-close"].rolling(20).mean()[-1]:
            signal="sell"

    return signal



In [7]:
def long_short(instrument):
    account_id = "101-002-19512089-001"
    r=trades.OpenTrades(accountID=account_id)
    open_trades=client.request(r)["trades"]
    l_s=""
    for i in range(len(open_trades)):
        if open_trades[i]["instrument"]==instrument:
            
            if int(open_trades[i]["currentUnits"])>0:
                l_s="long"

            elif int(open_trades[i]["currentUnits"])<0:
                l_s="short"

    return l_s


In [8]:
def main():
    global tickers
    for ticker in tickers:
        print("analyzzing for ",ticker)
        l_s= long_short(ticker)
        print(l_s)
        signal=trade_signal(ticker,l_s)
        if signal=="buy":
            market_order(ticker,200,.1)
            print("New long position initiated for ",ticker)

        if signal=="sell":
            market_order(ticker,-1*200,.1)
            print("New short position initiated for ", ticker)
    
   
            



In [9]:
start_time=time.time()
timeout=start_time+60*60*1
while time.time()<=timeout:
    try: 
        print("passthrough at ",time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())))
        main()
        time.sleep(60-(time.time()-start_time)%60)

    except KeyboardInterrupt:
        print("keyboard interuption ....... exiting")


passthrough at  2021-06-07 11:39:19
analyzzing for  EUR_USD

took data for  EUR_USD
analyzzing for  AUD_USD

took data for  AUD_USD
analyzzing for  GBP_USD
short
took data for  GBP_USD
analyzzing for  USD_JPY
short
took data for  USD_JPY
analyzzing for  EUR_JPY
short
took data for  EUR_JPY
passthrough at  2021-06-07 11:40:19
analyzzing for  EUR_USD

took data for  EUR_USD
analyzzing for  AUD_USD

took data for  AUD_USD
analyzzing for  GBP_USD
short
took data for  GBP_USD
analyzzing for  USD_JPY
short
took data for  USD_JPY
analyzzing for  EUR_JPY
short
took data for  EUR_JPY
passthrough at  2021-06-07 11:41:19
analyzzing for  EUR_USD

took data for  EUR_USD
analyzzing for  AUD_USD

took data for  AUD_USD
analyzzing for  GBP_USD
short
took data for  GBP_USD
analyzzing for  USD_JPY
short
took data for  USD_JPY
analyzzing for  EUR_JPY
short
took data for  EUR_JPY
passthrough at  2021-06-07 11:42:19
analyzzing for  EUR_USD

took data for  EUR_USD
analyzzing for  AUD_USD

took data for  AUD